In [24]:
import torch
from torch import nn
from torch.quantization import get_default_qconfig, prepare, convert, fuse_modules
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Subset, random_split
# from Model.Schema import resModel
from tqdm import tqdm
from PIL import Image


from torch.quantization import QuantStub, DeQuantStub
import torchmetrics


In [25]:
class resModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.resnet18(weights="IMAGENET1K_V1")
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.precision = torchmetrics.Precision(task="multiclass", average="macro", num_classes=num_classes)
        self.recall = torchmetrics.Recall(task="multiclass", average="macro", num_classes=num_classes)
        self.f1score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        x = self.quant(x)  # Quantize the input
        x = self.model(x)
        x = self.dequant(x)  # Dequantize the output
        return x

In [26]:
num_classes = 70
model = resModel(num_classes)
model.load_state_dict(torch.load('Weights/model_13.pth'))
model.eval()


resModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [27]:
# backend = "fbgemm"
# model.qconfig = torch.quantization.get_default_qconfig(backend)
# torch.backends.quantized.engine = backend
# model_static_quantized = torch.quantization.prepare(model, inplace=False)
# model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)

In [28]:
# modules_to_fuse = [
#     ["model.layer1.0.conv1", "model.layer1.0.bn1", "model.layer1.0.relu"],
#     ["model.layer1.0.conv2", "model.layer1.0.bn2"],
#     ["model.layer1.1.conv1", "model.layer1.1.bn1", "model.layer1.1.relu"],
#     ["model.layer1.1.conv2", "model.layer1.1.bn2"],

#     ["model.layer2.0.conv1", "model.layer2.0.bn1", "model.layer2.0.relu"],
#     ["model.layer2.0.conv2", "model.layer2.0.bn2"],
#     ["model.layer2.0.downsample.0", "model.layer2.0.downsample.1"],
#     ["model.layer2.1.conv1", "model.layer2.1.bn1", "model.layer2.1.relu"],
#     ["model.layer2.1.conv2", "model.layer2.1.bn2"],

#     ["model.layer3.0.conv1", "model.layer3.0.bn1", "model.layer3.0.relu"],
#     ["model.layer3.0.conv2", "model.layer3.0.bn2"],
#     ["model.layer3.0.downsample.0", "model.layer3.0.downsample.1"],
#     ["model.layer3.1.conv1", "model.layer3.1.bn1", "model.layer3.1.relu"],
#     ["model.layer3.1.conv2", "model.layer3.1.bn2"],

#     ["model.layer4.0.conv1", "model.layer4.0.bn1", "model.layer4.0.relu"],
#     ["model.layer4.0.conv2", "model.layer4.0.bn2"],
#     ["model.layer4.0.downsample.0", "model.layer4.0.downsample.1"],
#     ["model.layer4.1.conv1", "model.layer4.1.bn1", "model.layer4.1.relu"],
#     ["model.layer4.1.conv2", "model.layer4.1.bn2"],
# ]

# for module_list in modules_to_fuse:
#     fuse_modules(model, module_list, inplace=True)

model.qconfig = get_default_qconfig('fbgemm')

model_prepared = prepare(model, inplace=True)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

full_dataset = datasets.ImageFolder(root='dataset/train', transform=transform)

num_calibration = int(0.1 * len(full_dataset))
num_rest = len(full_dataset) - num_calibration
calibration_dataset, _ = random_split(full_dataset, [num_calibration, num_rest])

calibration_loader = DataLoader(calibration_dataset, batch_size=32, shuffle=True)

model_prepared.eval()
with torch.no_grad():
    for data, _ in tqdm(calibration_loader, desc="Calibrating", leave=True):
        data = data.to('cpu')
        model_prepared(data)

        
model_prepared.to('cpu')

model_quantized = convert(model_prepared, inplace=False)

torch.save(model_quantized.state_dict(), 'Weights/model_quantized_13.pth')

Calibrating: 100%|█████████████████████████████████████████████████████████████████████| 25/25 [00:38<00:00,  1.53s/it]


In [29]:
# test_transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor()
# ])

# image_path = 'dataset/valid/American  Spaniel/07.jpg'
# image = Image.open(image_path).convert('RGB')

# image = test_transform(image).unsqueeze(0)


# model_quantized.eval()

# device = torch.device('cpu')
# model_quantized.to(device)
# image = image.to(device)

# with torch.no_grad():
#     output = model_quantized(image)
#     prediction = torch.argmax(output, dim=1)

# print(prediction.item())

In [30]:
# model_fp32 = resModel(num_classes=70)
# model_fp32.load_state_dict(torch.load("Weights/model_13.pth"))

# model_fp32.train()

# model_fp32.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')

# model_fp32_prepared = torch.quantization.prepare_qat(model_fp32)

# model_quantized = torch.quantization.convert(model_fp32_prepared)

# model_quantized.load_state_dict(torch.load('Weights/model_quantized_13.pth'))

model_quantized.eval()



resModel(
  (model): ResNet(
    (conv1): QuantizedConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.11172884702682495, zero_point=51, padding=(3, 3), bias=False)
    (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.19457167387008667, zero_point=79, padding=(1, 1), bias=False)
        (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.06815629452466965, zero_point=71, padding=(1, 1), bias=False)
        (bn2): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicB

In [31]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from Model.Schema import resModel
import wandb
from tqdm import tqdm

# wandb.init(
#     project="EML",
#     config={
#         "architecture": "PretrainedResNet18",
#         "dataset": "70 Dog Breeds",
#         "quantization": "Static"
#     }
# )

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
test_dataset = datasets.ImageFolder(root='dataset/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def test_quantized_model(model, test_loader, device):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc="Testing", leave=True):
            data, target = data.to(device), target.to(device)

            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    test_accuracy = 100 * correct / total
    return test_accuracy

device= torch.device('cpu')
# model_quantized.to(device)

accuracy = test_quantized_model(model_quantized, test_loader, device)
# wandb.log({"Test Accuracy": accuracy})
print(f"Test Accuracy of the quantized model: {accuracy}%")


# wandb.finish()

Testing:   0%|                                                                                  | 0/11 [00:00<?, ?it/s]


NotImplementedError: Could not run 'aten::add.out' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::add.out' is only available for these backends: [CPU, CUDA, Meta, MkldnnCPU, SparseCPU, SparseCUDA, SparseCsrCPU, SparseCsrCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode

In [ ]:
device= torch.device('cpu')
# model_quantized.to(device)

accuracy = test_quantized_model(model_quantized, test_loader, device)
# wandb.log({"Test Accuracy": accuracy})
print(f"Test Accuracy of the quantized model: {accuracy}%")


# wandb.finish()